In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cafa-6-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-6-protein-function-prediction/IA.tsv
/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta
/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset-taxon-list.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-6-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo


In [2]:
!pip install biopython > dev null
!pip install obonet > dev null

In [3]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from tqdm.auto import tqdm
from collections import Counter
import networkx as nx

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec

# Modeling
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

import warnings
import obonet
import gc

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [4]:
class config:
    # File Paths
    TRAIN_TERMS_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv'
    TRAIN_SEQUENCES_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta'
    TRAIN_TAXONOMY_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_taxonomy.tsv'
    TEST_SEQUENCES_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta'
    IA_PATH = '/kaggle/input/cafa-6-protein-function-prediction/IA.tsv'
    
    # EDA & Plotting
    COLORS = ['#221f1f', '#b20710', '#e50914', 'grey']
    BACKGROUND_COLOR = '#f5f6f6'
    
    # Modeling
    PROBABILITY_THRESHOLD = 0.02

In [5]:
print("Loading tabular datasets...")
train_terms_df = pd.read_csv(config.TRAIN_TERMS_PATH, sep='\t')
train_taxonomy_df = pd.read_csv(config.TRAIN_TAXONOMY_PATH, sep='\t', header=None, names=['EntryID', 'taxonomyID'])
ia_df = pd.read_csv(config.IA_PATH, sep='\t', names=['term', 'ia'])

def load_fasta_to_dataframe(file_path, is_train=True):
    records = []
    parser = SeqIO.parse(file_path, "fasta")
    for record in tqdm(parser, desc=f"Parsing {file_path.split('/')[-1]}"):
        entry_id = record.id.split('|')[1] if is_train and '|' in record.id else record.id.split()[0]
        records.append({'EntryID': entry_id, 'sequence': str(record.seq)})
    return pd.DataFrame(records)

print("\nLoading sequence datasets...")
train_sequences_df = load_fasta_to_dataframe(config.TRAIN_SEQUENCES_PATH, is_train=True)

print("Consolidating data into a master dataframe for EDA...")
protein_labels = train_terms_df.groupby('EntryID')['term'].apply(list).reset_index(name='labels')
train_df_eda = pd.merge(train_sequences_df, train_taxonomy_df, on='EntryID', how='left')
train_df_eda = pd.merge(train_df_eda, protein_labels, on='EntryID', how='inner')
train_df_eda['seq_length'] = train_df_eda['sequence'].str.len()
train_df_eda['num_labels'] = train_df_eda['labels'].str.len()
print("Data loading complete.")

Loading tabular datasets...

Loading sequence datasets...


Parsing train_sequences.fasta: 0it [00:00, ?it/s]

Consolidating data into a master dataframe for EDA...
Data loading complete.


In [6]:
def extract_sequence_features(seq):
    if not seq or len(seq) == 0: return np.zeros(85)
    aa_counts = Counter(seq)
    length = len(seq)
    aa_freq = np.array([aa_counts.get(aa, 0) / length for aa in 'ACDEFGHIKLMNPQRSTVWY'])
    hydrophobic = sum(aa_counts.get(aa, 0) for aa in 'AILMFWYV') / length
    charged = sum(aa_counts.get(aa, 0) for aa in 'DEKR') / length
    aa_weights = {'A': 89, 'C': 121, 'D': 133, 'E': 147, 'F': 165, 'G': 75, 'H': 155, 'I': 131, 'K': 146, 'L': 131, 'M': 149, 'N': 132, 'P': 115, 'Q': 146, 'R': 174, 'S': 105, 'T': 119, 'V': 117, 'W': 204, 'Y': 181}
    mol_weight = sum(aa_counts.get(aa, 0) * aa_weights.get(aa, 0) for aa in aa_counts)
    groups = {'hydrophobic': set('AILMFWYV'),'polar': set('STNQ'),'positive': set('RK'),'negative': set('DE'),'aromatic': set('FWY'),'aliphatic': set('ILV'),'small': set('ACDGNPSTV')}
    ctd_features = [sum(1 for aa in seq if aa in group_aas) / length for group_aas in groups.values()]
    dipeptides = Counter([seq[i:i+2] for i in range(length - 1)])
    top_dipeptides = ['AL', 'LA', 'LE', 'EA', 'AA', 'AS', 'SA', 'EL', 'LL', 'AE', 'SE', 'ES', 'GA', 'AG', 'VA', 'AV', 'LV', 'VL', 'LS', 'SL']
    dipeptide_freq = np.array([dipeptides.get(dp, 0) / (length-1 if length > 1 else 1) for dp in top_dipeptides])
    tripeptides = Counter([seq[i:i+3] for i in range(length - 2)])
    top_tripeptides = ['ALA', 'LEA', 'EAL', 'LAL', 'AAA', 'LLE', 'ELE', 'ALE', 'GAL', 'ASA', 'VLA', 'LAV', 'SLS', 'LSL', 'GLA', 'LAG', 'AVL', 'VLA', 'SLE', 'LES']
    tripeptide_freq = np.array([tripeptides.get(tp, 0) / (length-2 if length > 2 else 1) for tp in top_tripeptides])
    return np.concatenate([aa_freq, np.array([np.log1p(length), hydrophobic, charged, np.log1p(mol_weight)]), np.array(ctd_features), dipeptide_freq, tripeptide_freq])

print("Extracting features for all training proteins...")

X_train_list, y_train_proteins = [], []
protein_ids_in_scope = set(train_terms_df['EntryID'].unique())
train_sequences_dict = dict(zip(train_sequences_df['EntryID'], train_sequences_df['sequence']))
for pid, seq in tqdm(train_sequences_dict.items(), desc="Processing Train Sequences"):
    if pid in protein_ids_in_scope:
        X_train_list.append(extract_sequence_features(seq))
        y_train_proteins.append(pid)
X_train = np.array(X_train_list)
print(f"Training feature matrix shape: {X_train.shape}")

Extracting features for all training proteins...


Processing Train Sequences:   0%|          | 0/82404 [00:00<?, ?it/s]

Training feature matrix shape: (82404, 71)


In [7]:
print("Preparing labels and training models for each ontology...")
models, mlb_dict = {}, {}

# CORRECT MAPPING of ontology name to the code used in the dataframe
ontology_map = {'BPO': 'P', 'CCO': 'C', 'MFO': 'F'}

for aspect_name, aspect_code in ontology_map.items():
    print(f"\n--- Processing {aspect_name} ({aspect_code}) ---")
    
    # Filter terms for the current ontology using the CORRECT code
    ont_terms_df = train_terms_df[train_terms_df['aspect'] == aspect_code]
    
    # Continue if no terms are found for this ontology
    if ont_terms_df.empty:
        print(f"No terms found for {aspect_name}. Skipping model training.")
        continue
        
    protein_terms = ont_terms_df.groupby('EntryID')['term'].apply(list).to_dict()
    labels_list = [protein_terms.get(pid, []) for pid in y_train_proteins]
    
    mlb = MultiLabelBinarizer(sparse_output=True)
    y_train_ont = mlb.fit_transform(labels_list)
    
    print(f"Number of unique {aspect_name} terms: {y_train_ont.shape[1]}")
    
    # Robustness check: only train if there are labels
    if y_train_ont.shape[1] > 0:
        mlb_dict[aspect_code] = mlb
        model = OneVsRestClassifier(LogisticRegression(max_iter=500, solver='lbfgs', C=0.5, random_state=42), n_jobs=-1)
        model.fit(X_train, y_train_ont)
        models[aspect_code] = model
        print(f"Model for {aspect_name} trained successfully.")
    else:
        print(f"Skipping model training for {aspect_name} as no labels were binarized.")

Preparing labels and training models for each ontology...

--- Processing BPO (P) ---
Number of unique BPO terms: 16858
Model for BPO trained successfully.

--- Processing CCO (C) ---
Number of unique CCO terms: 2651
Model for CCO trained successfully.

--- Processing MFO (F) ---
Number of unique MFO terms: 6616
Model for MFO trained successfully.


In [8]:
print("\nLoading and processing test sequences for submission...")
test_sequences_df = load_fasta_to_dataframe(config.TEST_SEQUENCES_PATH, is_train=False)
test_protein_ids = test_sequences_df['EntryID'].tolist()
test_sequences_dict = dict(zip(test_sequences_df['EntryID'], test_sequences_df['sequence']))
BATCH_SIZE = 5000
submission_list = []

for i in tqdm(range(0, len(test_protein_ids), BATCH_SIZE), desc="Predicting on Test Set"):
    batch_pids = test_protein_ids[i:i+BATCH_SIZE]
    batch_seqs = [test_sequences_dict[pid] for pid in batch_pids]
    X_batch = np.array([extract_sequence_features(seq) for seq in batch_seqs])
    for aspect_code, model in models.items():
        mlb = mlb_dict[aspect_code]
        y_pred_proba = model.predict_proba(X_batch)
        for j, pid in enumerate(batch_pids):
            probs = y_pred_proba[j]
            candidate_indices = np.where(probs > config.PROBABILITY_THRESHOLD)[0]
            for idx in candidate_indices:
                submission_list.append((pid, mlb.classes_[idx], probs[idx]))

print(f"Generated {len(submission_list):,} total predictions.")
submission_df = pd.DataFrame(submission_list, columns=['Protein Id', 'GO Term Id', 'Prediction'])
print("Applying 1500 prediction limit per protein...")
submission_df = submission_df.sort_values(by=['Protein Id', 'Prediction'], ascending=[True, False])
final_submission_df = submission_df.groupby('Protein Id').head(1500).reset_index(drop=True)
final_submission_df.to_csv('submission.tsv', sep='\t', index=False, header=False)

print("\nSubmission file 'submission.tsv' created successfully.")
print(f"Total predictions in final submission: {len(final_submission_df):,}")
print("Submission DataFrame Head:")
display(final_submission_df.head())


Loading and processing test sequences for submission...


Parsing testsuperset.fasta: 0it [00:00, ?it/s]

Predicting on Test Set:   0%|          | 0/45 [00:00<?, ?it/s]

Generated 4,192,941 total predictions.
Applying 1500 prediction limit per protein...

Submission file 'submission.tsv' created successfully.
Total predictions in final submission: 4,192,941
Submission DataFrame Head:


,Protein Id,GO Term Id,Prediction
0,A0A017SE81,GO:0005515,0.230935
1,A0A017SE81,GO:0005886,0.191714
2,A0A017SE81,GO:0005829,0.120174
3,A0A017SE81,GO:0005739,0.091896
4,A0A017SE81,GO:0005634,0.067264
